<a href="https://colab.research.google.com/github/yanzhi844517617/Mask_RCNN/blob/master/luggage_detect_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Luggage_detection_demo



This demo shows how we use YOLO to detection airport luggage.
First,We compare the performance of YOLOv5,YOLOv6 and YOLOv7 on airport luggage recognition，Thus,we can get the best YOLO-version.
Second,We fine-tuning the hyperparameters of the best YOLO-mode,then we can get the best Recognition-Mode with the best hyperparameters and the best YOLO-version.
Third,we use the best Recognition-Mode conbine with BoT-Sort algorithm to track the airport luggage.
Last,we conbine with the best YOLo-Box and Sam2 algorithm to segmentate the image，then，we can get the unique object in one image.

1.   List item
2.   List item



# Enviroment Set-up

Set-up YOLOv5

In [7]:
%cd /content

/content


In [5]:
%cd /content
!git clone https://github.com/ultralytics/yolov5  # clone
%cd yolov5
%pip install -qr requirements.txt comet_ml  # install

import torch
import utils
display = utils.notebook_init()  # checks

YOLOv5 🚀 v7.0-390-g0797106a Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)


Setup complete ✅ (2 CPUs, 12.7 GB RAM, 33.1/112.6 GB disk)


set-up YOLOv6

In [3]:
%cd /content
!nvidia-smi
!git clone https://github.com/meituan/YOLOv6
%cd YOLOv6
%pip install -r requirements.txt

import cv2
import time
import random
import numpy as np
#import onnxruntime as ort
from PIL import Image
from pathlib import Path
from collections import OrderedDict,namedtuple

/content
Tue Dec 31 08:14:48 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   61C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------

set-up YOLOv7

In [4]:
%cd /content
import sys
import torch
print(f"Python version: {sys.version}, {sys.version_info} ")
print(f"Pytorch version: {torch.__version__} ")

!nvidia-smi
!# Download YOLOv7 code
!git clone https://github.com/WongKinYiu/yolov7
%cd yolov7
!ls
!# Download trained weights
!wget https://github.com/WongKinYiu/yolov7/releases/download/v0.1/yolov7.pt

/content
Python version: 3.10.12 (main, Nov  6 2024, 20:22:13) [GCC 11.4.0], sys.version_info(major=3, minor=10, micro=12, releaselevel='final', serial=0) 
Pytorch version: 2.5.1+cu121 
Tue Dec 31 08:15:09 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   58C    P8              10W /  70W |      0MiB / 15360M

Import data in google drive

In [6]:
%cd /content
from google.colab import drive
drive.mount('/content/drive')

/content
Mounted at /content/drive


# Recognition with the pre-train mode

yolov5-test

In [ ]:
%cd /content/yolov5
!python detect.py --weights yolov5s.pt --img 640 --conf 0.25 --source /content/drive/MyDrive/coco/randam_image

yolov6-test

In [8]:
%cd /content/YOLOv6
!python tools/infer.py --weights yolov6s6.pt --img 640 640 --source /content/drive/MyDrive/coco/randam_image

/content/YOLOv6
Namespace(weights='yolov6s6.pt', source='/content/drive/MyDrive/coco/randam_image', webcam=False, webcam_addr='0', yaml='data/coco.yaml', img_size=[640, 640], conf_thres=0.4, iou_thres=0.45, max_det=1000, device='0', save_txt=False, not_save_img=False, save_dir=None, view_img=False, classes=None, agnostic_nms=False, project='runs/inference', name='exp', hide_labels=False, hide_conf=False, half=False)
checkpoint yolov6s6.pt not exist, try to downloaded it from github.
downloading url is: https://github.com/meituan/YOLOv6/releases/download/0.4.0/yolov6s6.pt, pealse make sure the version of the downloading model is correspoing to the code version!
checkpoint yolov6s6.pt downloaded and saved
Loading checkpoint from yolov6s6.pt
/content/YOLOv6/yolov6/utils/checkpoint.py:25: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which wi

yolov7-test

In [ ]:
%cd /content/yolov7
!python detect.py --weights yolov7.pt --conf 0.25 --img-size 640 --source /content/drive/MyDrive/coco/randam_image

# Train new mode with our data

Logger

In [7]:
#@title Select  logger {run: 'auto'}
logger = 'Comet' #@param ['Comet', 'ClearML', 'TensorBoard']

if logger == 'Comet':
  %pip install -q comet_ml
  import comet_ml; comet_ml.login()
elif logger == 'ClearML':
  %pip install -q clearml
  import clearml; clearml.browser_login()
elif logger == 'TensorBoard':
  %load_ext tensorboard
  %tensorboard --logdir runs/train

train new mode with yolov5

In [10]:
%cd /content/yolov5
!python train.py --img 640 --batch 16 --epochs 100 --data /content/drive/MyDrive/coco_x/data_x.yaml --weights yolov5s.pt --cache

/content/yolov5
2024-12-31 08:17:51.714316: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-31 08:17:51.733553: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-31 08:17:51.739366: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
train: weights=yolov5s.pt, cfg=, data=/content/drive/MyDrive/coco_x/data_x.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=100, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, evolve_population=data/hyps, resume_evolve=None, bucket=, cache=ram, image_weights=False, dev

train new mode with yolov6

In [8]:
%cd /content/YOLOv6
cuda = True
#python tools/train.py --batch 16 --epochs 100 --conf configs/yolov6s6_finetune.py --data /content/drive/MyDrive/coco_x/data_x.yaml --img 640 --device 0
!python tools/train.py --batch 16 --conf configs/yolov6s.py --epochs 100 --img-size 416 --data /content/drive/MyDrive/coco_x/data_x.yaml --device 0


/content/YOLOv6
2024-12-31 08:16:29.966000: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-31 08:16:29.983885: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-31 08:16:29.989162: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Using 1 GPU for training... 
training args are: Namespace(data_path='/content/drive/MyDrive/coco_x/data_x.yaml', conf_file='configs/yolov6s.py', img_size=416, rect=False, batch_size=16, epochs=100, workers=8, device='0', eval_interval=20, eval_final_only=False, heavy_eval_range=50, check_images=False, check_labels=False, output_dir='./runs/train',

train new mode with yolov7

In [9]:
%cd /content/yolov7
!python train.py --workers 8 --device 0 --batch-size 16 --epochs 100 --data /content/drive/MyDrive/coco_x/data_x.yaml --img 640 640 --cfg cfg/training/yolov7.yaml --weights '' --name yolov7 --hyp data/hyp.scratch.p5.yaml

/content/yolov7
2024-12-31 08:17:02.753754: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-31 08:17:02.771996: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-31 08:17:02.777288: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
YOLOR 🚀 v0.1-128-ga207844 torch 2.5.1+cu121 CUDA:0 (Tesla T4, 15102.0625MB)

Namespace(weights='', cfg='cfg/training/yolov7.yaml', data='/content/drive/MyDrive/coco_x/data_x.yaml', hyp='data/hyp.scratch.p5.yaml', epochs=100, batch_size=16, img_size=[640, 640], rect=False, resume=False, nosave=False, notest=False, noautoanchor=False, evolve=False, 

# Test with the new mode

yolov5 new-mode test

In [11]:
%cd /content/yolov5
!python detect.py --weights /content/yolov5/runs/train/exp/best.pt --img 640 --conf 0.25 --source /content/drive/MyDrive/coco/randam_image

/content/yolov5
Traceback (most recent call last):
  File "/content/yolov5/detect.py", line 48, in <module>
    from models.common import DetectMultiBackend
  File "/content/yolov5/models/common.py", line 39, in <module>
    from utils.dataloaders import exif_transpose, letterbox
  File "/content/yolov5/utils/dataloaders.py", line 23, in <module>
    import torchvision
  File "/usr/local/lib/python3.10/dist-packages/torchvision/__init__.py", line 10, in <module>
    from torchvision import _meta_registrations, datasets, io, models, ops, transforms, utils  # usort:skip
  File "/usr/local/lib/python3.10/dist-packages/torchvision/models/__init__.py", line 2, in <module>
    from .convnext import *
  File "/usr/local/lib/python3.10/dist-packages/torchvision/models/convnext.py", line 8, in <module>
    from ..ops.misc import Conv2dNormActivation, Permute
  File "/usr/local/lib/python3.10/dist-packages/torchvision/ops/__init__.py", line 23, in <module>
    from .poolers import MultiScaleRoIA

yolov6 new-mode test

In [12]:
%cd /content/yolov6
!python tools/infer.py  --img-size 640 --weights runs/train/exp/weights/best_ckpt.pt --source /content/drive/MyDrive/coco/randam_image --device 0

[Errno 2] No such file or directory: '/content/yolov6'
/content/yolov5
python3: can't open file '/content/yolov5/tools/infer.py': [Errno 2] No such file or directory


yolov7 new-mode test

In [13]:
%cd /content/yolov7
!python detect.py --weights /content/yolov7/runs/train/yolov7/weights/best.pt --conf 0.25 --img-size 640 --source /content/drive/MyDrive/coco/randam_image

/content/yolov7
Traceback (most recent call last):
  File "/content/yolov7/detect.py", line 10, in <module>
    from models.experimental import attempt_load
  File "/content/yolov7/models/experimental.py", line 6, in <module>
    from models.common import Conv, DWConv
  File "/content/yolov7/models/common.py", line 17, in <module>
    from utils.plots import color_list, plot_one_box
  File "/content/yolov7/utils/plots.py", line 15, in <module>
    import seaborn as sns
  File "/usr/local/lib/python3.10/dist-packages/seaborn/__init__.py", line 5, in <module>
    from .relational import *  # noqa: F401,F403
  File "/usr/local/lib/python3.10/dist-packages/seaborn/relational.py", line 21, in <module>
    from ._statistics import EstimateAggregator, WeightedAggregator
  File "/usr/local/lib/python3.10/dist-packages/seaborn/_statistics.py", line 32, in <module>
    from scipy.stats import gaussian_kde
  File "/usr/local/lib/python3.10/dist-packages/scipy/stats/__init__.py", line 606, in <mod

# train best hyperparameters

epochs=100，batch_size=8,16，32

In [ ]:
%cd /content/yolov5
!python train.py --img 640 --batch 8 --epochs 100 --data /content/drive/MyDrive/coco_x/data_x.yaml --weights yolov5s.pt --cache

In [ ]:
!python train.py --img 640 --batch 16 --epochs 100 --data /content/drive/MyDrive/coco_x/data_x.yaml --weights yolov5s.pt --cache

In [ ]:
!python train.py --img 640 --batch 32  --epochs 100 --data /content/drive/MyDrive/coco_x/data_x.yaml --weights yolov5s.pt --cache

# Conbine with BoT-Sort